# Deploy Web App on Azure Container Services (AKS)


In this notebook, we will set up an Azure Container Service which will be managed by Kubernetes. We will then take the Docker image we created earlier that contains our app and deploy it to the AKS cluster. Then, we will check everything is working by sending a question to it and getting it scored for matches in the original questions.

The process is split into the following steps:
- Define our resource names
- Login to Azure
- Create resource group and create AKS
- Connect to AKS
- Deploy our app

We assume that this notebook is running on Linux and Azure CLI is installed before proceeding.

In [ ]:
import os
import json
import dotenv
from utilities import write_json_to_file
%load_ext dotenv

## Setup

Determine the location of the dotenv file.

In [ ]:
dotenv_path = dotenv.find_dotenv()

Below are the various name definitions for the resources needed to setup AKS. Please modify the values in the `set_key` statements as you see fit.

In [ ]:
# If you have multiple subscriptions select the subscription you want to use 
dotenv.set_key(dotenv_path, 'selected_subscription', 'YOUR_SUBSCRIPTION')

# Resource group, name and location for AKS cluster
dotenv.set_key(dotenv_path, 'resource_group', 'RESOURCE_GROUP') 
dotenv.set_key(dotenv_path, 'aks_name', 'AKS_CLUSTER_NAME')
dotenv.set_key(dotenv_path, 'location', 'eastus')

Import the dotenv variables. Use them to determine the name of the image.

In [ ]:
%dotenv -o
image_name = os.getenv('docker_login') + os.getenv('image_repo')
image_name

In [ ]:
os.getenv('selected_subscription')

## Azure account login

The commands below will ensure you are logged in to your Azure account. If you are not logged in, it will initiate a login by popping up a browser at a login page where you will select your Azure account.

In [ ]:
%%bash
list=`az account list -o table`
if [ "$list" == '[]' ] || [ "$list" == '' ]; then 
  az login -o table
else
  az account list -o table 
fi

In [ ]:
!az account set --subscription "$selected_subscription"

In [ ]:
!az account show

You will also need to register the container service resources on your subscription if you haven't already done so.

In [ ]:
!az provider register -n Microsoft.ContainerService

In [ ]:
!az provider show -n Microsoft.ContainerService

## Create resources and dependencies

### Create resource group and AKS cluster

Azure encourages the use of groups to organize all the Azure components you deploy. That way it is easier to find them but also we can delete a number of resources simply by deleting the group.

In [ ]:
!az group create --name $resource_group --location $location

Below, we create the AKS cluster  with 5 nodes in the resource group we created earlier. This step can take ten or more minutes.

In [ ]:
%%time
!az aks create --resource-group $resource_group --name $aks_name --node-count 5 --generate-ssh-keys -s Standard_D4_v2

### Install kubectl CLI

To connect to the Kubernetes cluster, we will use kubectl, the Kubernetes command-line client. To install, run the following:

In [ ]:
!sudo env "PATH=$PATH" az aks install-cli

## Connect to AKS cluster

To configure kubectl to connect to the Kubernetes cluster, run the following command:

In [ ]:
!az aks get-credentials --resource-group $resource_group --name $aks_name

Let's verify connection by listing the nodes.

In [ ]:
!kubectl get nodes

Let's check the pods on our cluster.

In [ ]:
!kubectl get pods --all-namespaces

## Deploy application

Below we define our Kubernetes manifest file for our service and load balancer. Note that we have to specify the image name and cpu requests and limits for pods. We first start with  deploying 2 pods.

In [ ]:
app_template = {
  "apiVersion": "apps/v1beta1",
  "kind": "Deployment",
  "metadata": {
      "name": "azure-ml"
  },
  "spec":{
      "replicas":2,
      "template":{
          "metadata":{
              "labels":{
                  "app":"azure-ml"
              }
          },
          "spec":{
              "containers":[
                  {
                      "name": "azure-ml",
                      "image": image_name,

                      "ports":[
                          {
                              "containerPort":80,
                              "name":"model"
                          }
                      ],
                      "resources":{
                           "requests":{
                               "cpu": 1
                           },
                           "limits":{
                               "cpu": 1.25
                           }
                       }  
                  }
              ]
          }
      }
  }
}

service_temp = {
  "apiVersion": "v1",
  "kind": "Service",
  "metadata": {
      "name": "azure-ml"
  },
  "spec":{
      "type": "LoadBalancer",
      "ports":[
          {
              "port":80
          }
      ],
      "selector":{
            "app":"azure-ml"
      }
   }
}

In [ ]:
write_json_to_file(app_template, 'az-ml.json')

In [ ]:
write_json_to_file(service_temp, 'az-ml.json', mode='a')

Let's check the manifest created.

In [ ]:
!cat az-ml.json

Next, we will use kubectl create command to deploy our application.

In [ ]:
!kubectl create -f az-ml.json

Let's check if the pod is deployed. It can take several minutes for the deployment to be ready and running.

In [ ]:
!kubectl get pods --all-namespaces

If anything goes wrong you can use the commands below to observe the events on the node as well as review the logs.

In [ ]:
!kubectl get events

Check the logs for the first application pod.

In [ ]:
pod_json = !kubectl get pods -o json
pod_dict = json.loads(''.join(pod_json))

In [ ]:
!kubectl logs {pod_dict['items'][0]['metadata']['name']}

In [ ]:
!kubectl get deployment

It can take a few minutes for the service to populate the EXTERNAL-IP field below. This will be the IP you use to call the service. You can also specify an IP to use, please see the AKS documentation for further details.

In [ ]:
!kubectl get service azure-ml

# Scaling

In this part, we scale the number of pods to make sure we fully utilize the AKS cluster.

In [ ]:
!kubectl scale --current-replicas=2 --replicas=35 deployment/azure-ml

It can take a couple of minutes for all replicas to be running.

In [ ]:
!kubectl get pods --all-namespaces

In [ ]:
!kubectl get deployment

Next, we will [test our web application deployed on AKS](06_Test_WebApp.ipynb).

Once, you are done with all the notebooks of the tutorial, you can use the instructions in the [last notebook](09_Tear_Down.ipynb) to tear down the cluster.